In [15]:
# split train set and validation set for single model testing

import os
import shutil
import numpy as np

path = './data/'
if not os.path.exists(path+'image_train/'):
    os.mkdir(path+'image_train/')
if not os.path.exists(path+'image_val/'):
    os.mkdir(path+'image_val/')


filename_list = []
for d in os.listdir(path+'image/'):
    if not os.path.exists(path+'image_train/'+d):
        os.mkdir(path+'image_train/'+d)
    if not os.path.exists(path+'image_val/'+d):
        os.mkdir(path+'image_val/'+d)
    
    filename = []
    for file in os.listdir(path+'image/'+d):
        filename.append(d+'/'+file)
    filename_list.append(filename)

for i in range(len(filename_list)):
    np.random.shuffle(filename_list[i])

    
split_rate = 0.8
for i in range(len(filename_list)):
    for f in filename_list[i][:int(len(filename_list[i])*split_rate)]:
        shutil.copyfile(path+'image/'+f,path+'image_train/'+f)
    for f in filename_list[i][int(len(filename_list[i])*split_rate):]:
        shutil.copyfile(path+'image/'+f,path+'image_val/'+f)
        

In [11]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

X_train = []
X_test = []
X_train2 = []
X_test2 = []

path = './model/'
change = ['zca_whitening=True','zoom_range=0.2','shear_range=0.2','channel_shift_range=0.3']

for c in change:
    for filename in ["gap_"+c+"_ResNet50.h5", "gap_"+c+"_Xception.h5", "gap_"+c+"_InceptionV3.h5"]:
        with h5py.File(path+filename, 'r') as h:
            X_train2.append(np.array(h['train']))
            X_test2.append(np.array(h['test']))
            y_train = np.array(h['label'])

    X_train2 = np.concatenate(X_train, axis=1)
    X_test2 = np.concatenate(X_test, axis=1)
    X_train.append(X_train2)
    X_test.append(X_test2)

X_train = np.concatenate(X_train, axis=0)
X_test = np.concatenate(X_test, axis=0)

index = [i for i in range(0,len(y_train))]
import random
random.shuffle(index)

X_train = X_train[index]
y_train = y_train[index]

from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, num_classes=None)


from keras.models import *
from keras.layers import *

input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(0.5,name="dropout")(x)
x = Dense(30, activation='softmax',name="Dense_1")(x)
model = Model(input_tensor, x)

def log_loss(y_true, y_pred):
    re = .0
    for y in range(30):
        re += y_true[y]*tf.log(y_pred[y])
    return -re

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy',log_loss])

model.fit(X_train, y_train, batch_size=128, epochs=20, validation_split=0.2)
model.save(path+'model.h5')


y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)


Found 5340 images belonging to 30 classes.
Found 3000 images belonging to 1 classes.
Found 5340 images belonging to 30 classes.
Found 3000 images belonging to 1 classes.
Found 5340 images belonging to 30 classes.
Found 3000 images belonging to 1 classes.
Found 5340 images belonging to 30 classes.
Found 3000 images belonging to 1 classes.
Found 5340 images belonging to 30 classes.
Found 3000 images belonging to 1 classes.
Found 5340 images belonging to 30 classes.
Found 3000 images belonging to 1 classes.
Found 5340 images belonging to 30 classes.
Found 3000 images belonging to 1 classes.
Found 5340 images belonging to 30 classes.
Found 3000 images belonging to 1 classes.
Found 5340 images belonging to 30 classes.
Found 3000 images belonging to 1 classes.
Found 5340 images belonging to 30 classes.
Found 3000 images belonging to 1 classes.
Found 5340 images belonging to 30 classes.
Found 3000 images belonging to 1 classes.
Found 5340 images belonging to 30 classes.
Found 3000 images belo

In [9]:
# -*- encoding=utf-8 -*-
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *


import h5py

path='./data/'

def write_gap(MODEL, image_size, change, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator(change)
    train_generator = gen.flow_from_directory(path+"image", image_size, shuffle=False,
                                              batch_size=16)
    test_generator = gen.flow_from_directory(path+"test_A", image_size, shuffle=False,
                                             batch_size=16, class_mode=None)

    train = model.predict_generator(train_generator)
    test = model.predict_generator(test_generator)
    with h5py.File("./model/gap_"+change+"_%s.h5"%MODEL.func_name) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

#进行配置，使用30%的GPU
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7
session = tf.Session(config=config)

# 设置session
KTF.set_session(session)

change = ['zca_whitening=True','zoom_range=0.2','shear_range=0.2','channel_shift_range=0.3']

change = 'zca_whitening=True'
write_gap(ResNet50, (224, 224),change)
write_gap(Xception, (299, 299),change, xception.preprocess_input)
write_gap(InceptionV3, (299, 299),change, inception_v3.preprocess_input)

change = 'zoom_range=0.2'
write_gap(ResNet50, (224, 224),change)
write_gap(Xception, (299, 299),change, xception.preprocess_input)
write_gap(InceptionV3, (299, 299),change, inception_v3.preprocess_input)

change = 'shear_range=0.2'
write_gap(ResNet50, (224, 224),change)
write_gap(Xception, (299, 299),change, xception.preprocess_input)
write_gap(InceptionV3, (299, 299),change, inception_v3.preprocess_input)

change = 'channel_shift_range=0.3'
write_gap(ResNet50, (224, 224),change)
write_gap(Xception, (299, 299),change, xception.preprocess_input)
write_gap(InceptionV3, (299, 299),change, inception_v3.preprocess_input)


import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

X_train = []
X_test = []

path = './model/'

for filename in ["gap_ResNet50.h5", "gap_Xception.h5", "gap_InceptionV3.h5"]:
    with h5py.File(path+filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

index = [i for i in range(0,len(y_train))]
import random
random.shuffle(index)

X_train = X_train[index]
y_train = y_train[index]

from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, num_classes=None)


from keras.models import *
from keras.layers import *

input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(0.5,name="dropout")(x)
x = Dense(30, activation='softmax',name="Dense_1")(x)
model = Model(input_tensor, x)

def log_loss(y_true, y_pred):
    re = .0
    for y in range(30):
        re += y_true[y]*tf.log(y_pred[y])
    return -re

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy',log_loss])

model.fit(X_train, y_train, batch_size=128, epochs=20, validation_split=0.2)
model.save(path+'model.h5')


y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)


Found 5340 images belonging to 30 classes.
Found 3000 images belonging to 1 classes.


KeyboardInterrupt: 

In [4]:
import h5py
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from tensorflow.python.ops import math_ops
np.random.seed(2017)

X_train = []
X_test = []

path = './model/'
batchsize=128

for filename in ["gap_ResNet50.h5", "gap_Xception.h5", "gap_InceptionV3.h5"]:
    with h5py.File(path+filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

index = [i for i in range(0,len(y_train))]
import random
random.shuffle(index)

X_train = X_train[index]
y_train = y_train[index]

from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, num_classes=None)


from keras.models import *
from keras.layers import *

input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(0.5,name="dropout")(x)
x = Dense(1024,activation='relu')(x)
x = Dense(30, activation='softmax',name="Dense_1")(x)
model = Model(input_tensor, x)

# 仿造tf.losses.log_loss写的
def log_loss(y_true, y_pred):
    epsilon = 1e-7
    predictions = math_ops.to_float(y_pred)
    labels = math_ops.to_float(y_true)
    predictions.get_shape().assert_is_compatible_with(labels.get_shape())
    return -(math_ops.reduce_sum(math_ops.multiply(labels,math_ops.log(predictions)+epsilon)))/batchsize

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy',log_loss])

model.fit(X_train, y_train, batch_size=128, epochs=100, validation_split=0.3)

model.save(path+'model.h5')


Train on 61994 samples, validate on 26569 samples
Epoch 1/100
61994/61994 [==============================] - 43s 701us/step - loss: 1.3764 - acc: 0.8536 - log_loss: 2.5611 - val_loss: 1.0910 - val_acc: 0.9279 - val_log_loss: 2.1995
Epoch 2/100
61994/61994 [==============================] - 42s 685us/step - loss: 1.0843 - acc: 0.9187 - log_loss: 2.0807 - val_loss: 0.5664 - val_acc: 0.9618 - val_log_loss: 0.9393
Epoch 3/100
61994/61994 [==============================] - 42s 683us/step - loss: 0.5790 - acc: 0.9517 - log_loss: 1.1861 - val_loss: 0.5528 - val_acc: 0.9647 - val_log_loss: 1.1915
Epoch 4/100
61994/61994 [==============================] - 42s 685us/step - loss: 0.5715 - acc: 0.9546 - log_loss: 1.2049 - val_loss: 0.5521 - val_acc: 0.9651 - val_log_loss: 1.1004
Epoch 5/100
61994/61994 [==============================] - 42s 682us/step - loss: 0.5651 - acc: 0.9561 - log_loss: nan - val_loss: 0.5534 - val_acc: 0.9644 - val_log_loss: 1.0713
Epoch 6/100
61994/61994 [==================

KeyboardInterrupt: 

In [191]:
model.load_weights(path+'model.h5')
model.fit(X_train, y_train, batch_size=128, epochs=100, validation_split=0.2)

Train on 70850 samples, validate on 17713 samples
Epoch 1/100
70850/70850 [==============================] - 7s 94us/step - loss: 0.0030 - acc: 0.9994 - log_loss: 0.0033 - val_loss: 3.9086e-04 - val_acc: 1.0000 - val_log_loss: 3.9031e-04
Epoch 2/100
70850/70850 [==============================] - 7s 93us/step - loss: 0.0028 - acc: 0.9994 - log_loss: 0.0026 - val_loss: 4.0505e-04 - val_acc: 1.0000 - val_log_loss: 4.0080e-04
Epoch 3/100
70850/70850 [==============================] - 7s 93us/step - loss: 0.0026 - acc: 0.9996 - log_loss: 0.0025 - val_loss: 4.1200e-04 - val_acc: 0.9999 - val_log_loss: 3.6917e-04
Epoch 4/100
70850/70850 [==============================] - 7s 94us/step - loss: 0.0028 - acc: 0.9996 - log_loss: 0.0029 - val_loss: 3.9943e-04 - val_acc: 1.0000 - val_log_loss: 4.1001e-04
Epoch 5/100
70850/70850 [==============================] - 7s 94us/step - loss: 0.0026 - acc: 0.9996 - log_loss: 0.0024 - val_loss: 3.3984e-04 - val_acc: 1.0000 - val_log_loss: 3.7367e-04
Epoch 6/10

70850/70850 [==============================] - 7s 94us/step - loss: 0.0022 - acc: 0.9995 - log_loss: 0.0023 - val_loss: 1.9487e-04 - val_acc: 1.0000 - val_log_loss: 2.2022e-04
Epoch 88/100
70850/70850 [==============================] - 7s 94us/step - loss: 0.0019 - acc: 0.9997 - log_loss: 0.0018 - val_loss: 2.0000e-04 - val_acc: 1.0000 - val_log_loss: 2.7786e-04
Epoch 89/100
70850/70850 [==============================] - 7s 94us/step - loss: 0.0021 - acc: 0.9995 - log_loss: 0.0020 - val_loss: 2.2284e-04 - val_acc: 1.0000 - val_log_loss: 2.6999e-04
Epoch 90/100
70850/70850 [==============================] - 7s 94us/step - loss: 0.0019 - acc: 0.9996 - log_loss: 0.0018 - val_loss: 2.0227e-04 - val_acc: 1.0000 - val_log_loss: 2.5969e-04
Epoch 91/100
70850/70850 [==============================] - 7s 94us/step - loss: 0.0020 - acc: 0.9996 - log_loss: 0.0019 - val_loss: 2.0090e-04 - val_acc: 1.0000 - val_log_loss: 2.3071e-04
Epoch 92/100
70850/70850 [==============================] - 7s 94us/

In [5]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

3000/3000 [==============================] - 1s 187us/step


In [3]:
# -*- encoding=utf-8 -*-
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from tensorflow.python.ops import math_ops
from keras.applications.inception_resnet_v2 import preprocess_input

import keras.backend.tensorflow_backend as KTF
import tensorflow as tf

#进行配置，使用30%的GPU
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 1
session = tf.Session(config=config)

# 设置session
KTF.set_session(session)

path = './data/'

import h5py

batchsize = 64
image_size = (299,299)
lambda_func = preprocess_input

width = image_size[0]
height = image_size[1]
input_tensor = Input((height, width, 3))
x = input_tensor
if lambda_func:
    x = Lambda(lambda_func)(x)

base_model = InceptionResNetV2(input_tensor=x, include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(30, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

model.load_weights(path+'./data/InceptionResNetV2_e50.h5')
y_pred = model.predict(X_test, verbose=1)

def getResult(output_path,y_pred):
    import os
    import pandas as pd
    
    path = './data/test_A/test'
    filename = []
    for file in os.listdir(path):
        filename.append(file)

    f = []
    for s in filename:
        for i in range(30):
            f.append(s.split('.')[0])

    index = [i for i in range(1,31)]*len(filename)

    y_pred = y_pred.clip(min=0.003, max=0.995)

    import numpy as np
    def softmax(x):
        """Compute the softmax of vector x."""
        exp_x = np.exp(x)
        softmax_x = exp_x / np.sum(exp_x)
        return softmax_x

    for y in range(len(y_pred)):
        y_pred[y] = softmax(y_pred[y])


    prob = [list(y_pred[j]) for j in range(len(y_pred))]
    probs = []
    for p in prob:
        probs.extend(p)

    pd.DataFrame({'f':f,'index':index,'probs':probs}).to_csv(output_path,header=None,index=None)    
    print(max(pd.read_csv(output_path,header=None).iloc[:,2]))

getResult('result/incepV4_e50.csv',y_pred)

KeyboardInterrupt: 